<a href="https://colab.research.google.com/github/Belvinbarasa/Eng.barasa/blob/main/FINE_TUNING.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()  # Upload kaggle.json here


Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"belvinbarasa","key":"e25b415e9d27e487a8bb0f5d40a1f9cf"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets download -d praveengovi/covid19engineeringbooksnlpdataset


Dataset URL: https://www.kaggle.com/datasets/praveengovi/covid19engineeringbooksnlpdataset
License(s): CC-BY-SA-4.0
^C


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip covid19engineeringbooksnlpdataset.zip -d engineering_books


unzip:  cannot find or open covid19engineeringbooksnlpdataset.zip, covid19engineeringbooksnlpdataset.zip.zip or covid19engineeringbooksnlpdataset.zip.ZIP.


In [ ]:
import os

combined_text_path = "engineering_corpus.txt"
with open(combined_text_path, "w", encoding="utf-8") as outfile:
    for root, _, files in os.walk("engineering_books"):
        for fname in files:
            if fname.endswith(".txt"):
                with open(os.path.join(root, fname), "r", encoding="utf-8", errors="ignore") as infile:
                    outfile.write(infile.read() + "\n")


In [ ]:
!pip install transformers datasets accelerate


In [ ]:
!ls -l engineering_corpus.txt

-rw-r--r-- 1 root root 0 Apr 29 10:06 engineering_corpus.txt


In [ ]:
dataset = get_dataset(combined_text_path, tokenizer, block_size=64)

/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
print(len(dataset))

0


In [ ]:
with open("engineering_corpus.txt", "r", encoding="utf-8") as f:
          content = f.read()
          print(f"File size: {len(content)} bytes")
          if not content.strip():  # Check if content is only whitespace
              print("File appears to be empty or contain only whitespace.")

File size: 0 bytes
File appears to be empty or contain only whitespace.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
import os

combined_text_path = "engineering_corpus.txt" # Removed extra indent here
with open(combined_text_path, "w", encoding="utf-8") as outfile:
    for root, _, files in os.walk("engineering_books"):
        for fname in files:
            if fname.endswith(".txt"):
                filepath = os.path.join(root, fname)
                print(f"Processing: {filepath}") # Debug print
                try:
                    with open(filepath, "r", encoding="utf-8", errors="ignore") as infile:
                        content = infile.read()
                        outfile.write(content + "\n")
                        print(f"Read {len(content)} bytes from {filepath}") # Debug print
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments

# Load tokenizer and model
tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained("distilgpt2")

# Prepare dataset
def get_dataset(file_path, tokenizer, block_size=128):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )

dataset = get_dataset(combined_text_path, tokenizer)
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

# Set training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-engineering",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=1,
    logging_steps=100,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

# Train!
trainer.train()


/usr/local/lib/python3.11/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
trainer.save_model("./gpt2-engineering")
tokenizer.save_pretrained("./gpt2-engineering")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')